In [ ]:
import numpy as np
import yaml
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, PreText, DataTable, Select
from bokeh.plotting import figure, curdoc
from bokeh.themes import Theme

from pytmatrix.tmatrix import Scatterer
from pytmatrix.psd import PSDIntegrator, GammaPSD
from pytmatrix import orientation, radar, tmatrix_aux, refractive

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
from scipy.special import gamma


In [24]:
# calculations
def drop_ar(D_eq):
    if D_eq < 0.7:
        return 1.0;
    elif D_eq < 1.5:
        return 1.173 - 0.5165*D_eq + 0.4698*D_eq**2 - 0.1317*D_eq**3 - \
            8.5e-3*D_eq**4
    else:
        return 1.065 - 6.25e-2*D_eq - 3.99e-3*D_eq**2 + 7.66e-4*D_eq**3 - \
            4.095e-5*D_eq**4 


def get_scattering_props(Dm=2.0,logNw=3.0,mu=0.0,wavelength='10',cant=0.0,ptype='rain'):
    if ((wavelength == '5') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_C, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '3') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_X, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '10') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_S, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '5') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_C, m=refractive.m_w_10C[tmatrix_aux.wl_C])
    elif ((wavelength == '3') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_X, m=refractive.m_w_10C[tmatrix_aux.wl_X])
    elif ((wavelength == '10') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_S, m=refractive.m_w_10C[tmatrix_aux.wl_S])
    scatterer.psd_integrator = PSDIntegrator()
    if ptype == 'rain':
        scatterer.psd_integrator.axis_ratio_func = lambda D: 1.0/drop_ar(D)
    if ptype == 'hail':
        scatterer.psd_integrator.axis_ratio_func = lambda D: 0.99
    scatterer.psd_integrator.D_max = 10.0
    scatterer.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)
    if cant > 0.0:
        scatterer.or_pdf = orientation.gaussian_pdf(cant)
    scatterer.orient = orientation.orient_averaged_fixed
    scatterer.psd_integrator.init_scatter_table(scatterer)

    D0 = (3.67 + mu)/(4 + mu) * Dm
    Nw = 10.**logNw   
    scatterer.psd = GammaPSD(D0=D0, Nw=Nw, mu=mu)

    Zh = 10*np.log10(radar.refl(scatterer))
    Zv = 10*np.log10(radar.refl(scatterer, False))
    Zdr = 10.*np.log10(radar.Zdr(scatterer))
    Ldr = 10*np.log10(radar.ldr(scatterer))
    rho_hv = radar.rho_hv(scatterer)
    delta = radar.delta_hv(scatterer)
    scatterer.set_geometry(tmatrix_aux.geom_horiz_forw)
    Kdp = radar.Kdp(scatterer)
    Ah = radar.Ai(scatterer)
    Av = radar.Ai(scatterer,h_pol=False)

    f_u = (6/(4**4))*((4 + mu)**(mu + 4))/(gamma(mu+4))

    dsd_df = calc_dsd(Dm,logNw,mu)

    NT = Nw * f_u * gamma(mu + 1) * Dm / ((4 + mu)**(mu + 1))
    if ptype == 'rain':
        LWC = (1. * np.pi * Nw * Dm**4) / (4.**4 * 1000.)
    elif ptype == 'hail':
        LWC = (1. * np.pi * Nw * Dm**4) / (4.**4 * 917.)
        
#    Vt = -0.193 + 4.96*dsd_df['D'] - 0.904*dsd_df['D']**2 + 0.0566*dsd_df['D']**3
    
    Vt = 17.67 * dsd_df['D']**0.67
    
    R = np.sum(dsd_df['ND']*(np.pi*dsd_df['D']/6.)*Vt)/3600.



    integ_df = pd.DataFrame({'NT':[NT], 
                            'WC (g m-3)':[LWC], 
                            'Zh (dBZ)':[Zh], 
                            'Zv (dBZ)':[Zv], 
                            'Zdr (dB)': [Zdr], 
                            'Ldr (dB)': [Ldr], 
                            'rho_hv':[rho_hv], 
                            'Kdp (deg km-1)':[Kdp], 
                            'delta (deg km-1)':[delta],
                            'Ah (dBZ/km)':[Ah],
                            'Adr (dB/km)':[Ah-Av],
                            'R (mm/hr)': R})

    return dsd_df, integ_df



def calc_dsd(Dm,logNw,mu):
    f_u = (6/4**4)*((4 + mu)**(mu + 4))/(gamma(mu+4))
    dsd_df = pd.DataFrame()
    dsd_df['D'] = np.arange(0.1,20,0.1)
    dsd_df['ND'] = 10**logNw * f_u * ((dsd_df['D'] / Dm) ** mu) * np.exp(-1.*(4 + mu)*(dsd_df['D'] / Dm))
 
    return dsd_df


In [29]:
# Set up data
Dm = 1.5
logNw = np.log10(5800)
mu = 3
wavelength = '10'
cant = 0.0
ptype = 'rain'

dsd_df, integ_df = get_scattering_props(Dm=Dm, logNw=logNw, mu=mu, wavelength=wavelength, 
    cant=cant, ptype=ptype)

In [ ]:
integ_df

In [ ]:
!wget https://github.com/maahn/pyOptimalEstimation_examples/raw/master/data/huntsville_parameters.nc

In [7]:
import xarray as xr
data = xr.open_dataset('huntsville_parameters.nc')

In [ ]:
data.Nw.max

In [1]:
def calc_dsd_all(Dm,Nw):
    dsd_df, integ_df = get_scattering_props(Dm=float(data.Dm[i]), logNw=float(np.log10(data.Nw[i])), 
                                            mu=np.random.normal(3,0.5), wavelength=wavelength, 
                                            cant=cant, ptype=ptype)
    return integ_df

In [ ]:
from dask.distributed import Client
client = Client() 

In [ ]:
client

In [ ]:
futures = client.map(calc_dsd_all,data.Dm.values,np.log10(data.Nw.values))

In [ ]:
futures

In [ ]:
all_df = pd.DataFrame()
for i in range(data.time.size):
    print(i)
    dsd_df, integ_df = get_scattering_props(Dm=float(data.Dm[i]), logNw=float(np.log10(data.Nw[i])), 
                                            mu=np.random.normal(3,0.5), wavelength=wavelength, 
                                            cant=cant, ptype=ptype)
    integ_df.to_csv('~/Downloads/out/scatt_{:05d}.csv'.format(i))

In [ ]:
float(data.Dm[i])